In [ ]:
import requests
import lxml.html as lh
import pandas as pd
from bs4 import BeautifulSoup
from pprint import pprint
import re
import time
from selenium import webdriver
from selenium.webdriver.firefox.firefox_binary import FirefoxBinary 
from selenium.webdriver.common.keys import Keys   # For keyboard keys 
from selenium.webdriver.chrome.options import Options
import datetime
import argparse
import os
from selenium.webdriver.support.ui import WebDriverWait
from urllib.parse import urlparse,urlsplit,parse_qs,parse_qsl,urlencode,urlunparse
from pathlib import Path
from selenium.common.exceptions import NoSuchElementException



In [ ]:
chrome_options = Options()
path = "/home/ahmed/labs/isie/"
chrome_options.add_experimental_option("prefs", {
  "download.default_directory": path,
  "download.prompt_for_download": False,
  "download.directory_upgrade": True,
  "safebrowsing.enabled": True
})
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
locate_driver = '/usr/lib/chromium-browser/chromedriver'
driver = webdriver.Chrome(locate_driver,options=chrome_options)
#validés:
#http://www.isie.tn/resultats/resultats-presidentielles/election-presidentielle-1er-tour/ 
#http://www.isie.tn/elections/election-2011/
#http://www.isie.tn/elections/elections-municipales-2018/resultats/
#http://www.isie.tn/resultats/resultats-presidentielles/election-presidentielle-2eme-tour/
url = 'http://www.isie.tn/resultats/resultats-presidentielles/election-presidentielle-2eme-tour/'
#url_parse = urlparse(url)
driver.get(url)

In [ ]:
driver

In [ ]:
start_time = time.time()
driver.refresh();
time.sleep(1)
list_csv_link= []
#this class is the global class existed on all page 
html_list = driver.find_element_by_class_name("entry-content")
#check if buton plus exist because if existed we should click all folder to get link inside
try:
    plus_button=driver.find_element_by_class_name("treeview")
    plus_list_exist=True
except NoSuchElementException:
    print("No plus button found")
    plus_list_exist=False
links = html_list.find_elements_by_tag_name('a')
print('fetching all Files')
#loop on all link exist in html 
for num,link in enumerate(links):
    print(link.get_attribute("href"))
    #for check if buton plus exist
    if plus_list_exist:
            #for check if is link wich contain files 
        if '#' in link.get_attribute("href"):
                #print(link.text)
            link.click()
            time.sleep(0.5)            
    if link.get_attribute('href').endswith('.xlsx'):
        dicti = {'region':link.text,'link':link.get_attribute('href')}      
        list_csv_link.append(dicti)
#this because after links is opened in case button plus, we should refind all new links that appears        
if plus_list_exist:
    links = html_list.find_elements_by_tag_name('a')
    list_csv_link = [{'region':link.text,'link':link.get_attribute('href')} for link in links if link.get_attribute('href').endswith('.xlsx')]
print('we have traited ' + str(num) + ' link and files')
print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
list_csv_link

In [ ]:
len()

In [ ]:
#Download part and folder structuration
base_folder = os.path.basename(os.path.dirname(url))
if not os.path.exists(path+base_folder):
    for link in list_csv_link:
        if link.get('region') !='':
            filename =  link.get('region')
        else:
            file_name_start_pos = link.get('link').rfind("/") + 1
            filename = link.get('link')[file_name_start_pos:]
            link_path_fileless = filename
        if '%' in link.get('link'):
            file_name_start =link.get('link').rfind("/") 
            file_name = link.get('link')[file_name_start:]
            link_fileless = link.get('link').replace(file_name,"")
            folder_name_start_pos =link_fileless.rfind("/") + 1
            folder_name_from_path = link_fileless[folder_name_start_pos:]
            
            if '%' in folder_name_from_path:
                sub_folder = os.path.basename(os.path.dirname(link_fileless)) 
            else:
                sub_folder = os.path.basename(os.path.basename(urlparse(link_fileless).path))  
        #sub_folder = os.path.basename(os.path.basename(urlparse(os.path.dirname(os.path.dirname(link.get('link') ))).path)) 
        else:
            sub_folder = os.path.basename(os.path.dirname( link.get('link')))
        if not os.path.exists(path+base_folder+'/'+sub_folder):
            os.makedirs(path+base_folder+'/'+sub_folder)
        r = requests.get( link.get('link'), stream=True)
        if r.status_code == requests.codes.ok:
            with open(path+base_folder +'/'+sub_folder+'/'+filename, 'wb') as f:
                for data in r:
                    f.write(data)
    print('Done.')
else:
    print('folder ' +base_folder+' already exist, maybe you have donwloaded the files .')     